In [1]:
import pandas as pd
import numpy as np
from scipy.stats import f

In [2]:
fh = '/Users/henrycosentino/Desktop/Python/Projects/Options Strategy/Variance Test/stock_variance_pulls.xlsx'
worksheet_nm = ['UD Values', 'UR Values', 'SD Values', 'SR Values']

u_dates_df, u_returns_df, s_dates_df, s_returns_df = [pd.read_excel(fh, sheet_name=sheet) for sheet in worksheet_nm]

In [3]:
u_dates_df

,DUK US Equity,AEP US Equity,AES UN Equity,AVA UN Equity,BKH UN Equity,CMS US Equity,CNP UN Equity,D UN Equity,DTE US Equity,ED UN Equity,...,OGE US Equity,PCG UN Equity,PEG UN Equity,PNW UN Equity,PPL US Equity,RSG UN Equity,SO UN Equity,TXNM UN Equity,UGI UN Equity,WEC US Equity
0,19920423.0,19900424.0,19910331.0,19900331,19920722.0,19920731.0,19920506.0,19920124.0,19920427.0,19940125.0,...,19920428.0,19920415.0,19920721.0,19900423.0,19920428.0,19980730.0,19920428.0,19930202.0,19920428.0,19920723.0
1,19920723.0,19900724.0,19910630.0,19900630,19921021.0,19921022.0,19920803.0,19920417.0,19920727.0,19930427.0,...,19920724.0,19920716.0,19921020.0,19901105.0,19920721.0,19981029.0,19920728.0,19920424.0,19920727.0,19921028.0
2,19921022.0,19901023.0,19910930.0,19900930,19930128.0,19940125.0,19921026.0,19920717.0,19921026.0,19930727.0,...,19921026.0,19921021.0,19930120.0,19910423.0,19921028.0,19990128.0,19921028.0,19920721.0,19921022.0,19930127.0
3,19930125.0,19910122.0,19911231.0,19901231,19930427.0,19930422.0,19930203.0,19921019.0,19930126.0,19931026.0,...,19930128.0,19930120.0,19930420.0,19910805.0,19930127.0,19990429.0,19930118.0,19921023.0,19930217.0,19930428.0
4,19930423.0,19920416.0,19920421.0,19910331,19930726.0,19930722.0,19930428.0,19930122.0,19930426.0,19940125.0,...,19930426.0,19930421.0,19930720.0,19911104.0,19930428.0,19990728.0,19930428.0,19930203.0,19930427.0,19930728.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,NaN,20241106.0,20241031.0,20231101,NaN,NaN,20241106.0,NaN,NaN,NaN,...,NaN,NaN,NaN,20240801.0,20240801.0,NaN,20241108.0,NaN,NaN,NaN
135,NaN,NaN,NaN,20240221,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,20241106.0,20241106.0,NaN,NaN,NaN,NaN,NaN
136,NaN,NaN,NaN,20240501,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
137,NaN,NaN,NaN,20240807,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Function to clean dates frames

def dates_cleaner(frame):
    for col in frame.columns:
        if frame[col].dtype == 'object' or frame[col].dtype in ['int64', 'float64']:
            frame[col] = frame[col].apply(lambda row: (
                pd.to_datetime(str(int(row)), format='%Y%m%d', errors='coerce')
                if not pd.isna(row) else row
            ))
    return frame

In [5]:
# Cleaned dates frames

u_dates_df = dates_cleaner(u_dates_df)
s_dates_df = dates_cleaner(s_dates_df)

u_dates_df

,DUK US Equity,AEP US Equity,AES UN Equity,AVA UN Equity,BKH UN Equity,CMS US Equity,CNP UN Equity,D UN Equity,DTE US Equity,ED UN Equity,...,OGE US Equity,PCG UN Equity,PEG UN Equity,PNW UN Equity,PPL US Equity,RSG UN Equity,SO UN Equity,TXNM UN Equity,UGI UN Equity,WEC US Equity
0,1992-04-23,1990-04-24,1991-03-31,1990-03-31,1992-07-22,1992-07-31,1992-05-06,1992-01-24,1992-04-27,1994-01-25,...,1992-04-28,1992-04-15,1992-07-21,1990-04-23,1992-04-28,1998-07-30,1992-04-28,1993-02-02,1992-04-28,1992-07-23
1,1992-07-23,1990-07-24,1991-06-30,1990-06-30,1992-10-21,1992-10-22,1992-08-03,1992-04-17,1992-07-27,1993-04-27,...,1992-07-24,1992-07-16,1992-10-20,1990-11-05,1992-07-21,1998-10-29,1992-07-28,1992-04-24,1992-07-27,1992-10-28
2,1992-10-22,1990-10-23,1991-09-30,1990-09-30,1993-01-28,1994-01-25,1992-10-26,1992-07-17,1992-10-26,1993-07-27,...,1992-10-26,1992-10-21,1993-01-20,1991-04-23,1992-10-28,1999-01-28,1992-10-28,1992-07-21,1992-10-22,1993-01-27
3,1993-01-25,1991-01-22,1991-12-31,1990-12-31,1993-04-27,1993-04-22,1993-02-03,1992-10-19,1993-01-26,1993-10-26,...,1993-01-28,1993-01-20,1993-04-20,1991-08-05,1993-01-27,1999-04-29,1993-01-18,1992-10-23,1993-02-17,1993-04-28
4,1993-04-23,1992-04-16,1992-04-21,1991-03-31,1993-07-26,1993-07-22,1993-04-28,1993-01-22,1993-04-26,1994-01-25,...,1993-04-26,1993-04-21,1993-07-20,1991-11-04,1993-04-28,1999-07-28,1993-04-28,1993-02-03,1993-04-27,1993-07-28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,NaT,2024-11-06,2024-10-31,2023-11-01,NaT,NaT,2024-11-06,NaT,NaT,NaT,...,NaT,NaT,NaT,2024-08-01,2024-08-01,NaT,2024-11-08,NaT,NaT,NaT
135,NaT,NaT,NaT,2024-02-21,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,2024-11-06,2024-11-06,NaT,NaT,NaT,NaT,NaT
136,NaT,NaT,NaT,2024-05-01,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
137,NaT,NaT,NaT,2024-08-07,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


In [6]:
# Function to rename columns

def column_renamer(frame):
    new_col_name_ls = []
    ticker_ls = []
    count = 0

    for name in frame.columns:
        if name.endswith('Equity'):
            pos = name.find(' ')
            ticker = name[:pos]
            ticker_ls.append(ticker)
            dt_name = f"{ticker} Date"
            new_col_name_ls.append(dt_name)
        else:
            ticker = ticker_ls[count - 1]
            ticker_ls.append(ticker)
            rt_name = f"{ticker} Price"
            new_col_name_ls.append(rt_name)

    return new_col_name_ls

In [7]:
# Renaming columns

s_returns_df.columns = column_renamer(s_returns_df)
u_returns_df.columns = column_renamer(u_returns_df)

s_returns_df

,MO Date,MO Price,UVV Date,UVV Price,HSY Date,HSY Price,KR Date,KR Price,KMB Date,KMB Price,...,SAM Date,SAM Price,UNFI Date,UNFI Price,NUS Date,NUS Price,CALM Date,CALM Price,FDP Date,FDP Price
0,1997-10-24,9.1571,1997-10-24,37.5625,1997-10-24,27.4375,1997-10-24,8.1094,1997-10-24,49.9422,...,1997-10-24,9.938,1997-10-24,11.8125,1997-10-24,20.2500,1997-10-24,1.5625,1997-10-24,17.0000
1,1997-10-27,8.6219,1997-10-27,37.1875,1997-10-27,25.7813,1997-10-27,7.5469,1997-10-27,48.0576,...,1997-10-27,9.688,1997-10-27,10.8750,1997-10-27,19.6875,1997-10-27,1.5313,1997-10-27,16.0000
2,1997-10-28,9.1571,1997-10-28,37.6250,1997-10-28,26.8125,1997-10-28,7.5938,1997-10-28,48.6465,...,1997-10-28,10.000,1997-10-28,11.1250,1997-10-28,20.0000,1997-10-28,1.5625,1997-10-28,15.8125
3,1997-10-29,9.1426,1997-10-29,37.8750,1997-10-29,27.4375,1997-10-29,7.8594,1997-10-29,48.9999,...,1997-10-29,10.000,1997-10-29,11.4375,1997-10-29,19.7500,1997-10-29,1.5313,1997-10-29,16.0000
4,1997-10-30,8.9980,1997-10-30,37.0625,1997-10-30,27.5000,1997-10-30,7.9844,1997-10-30,48.1754,...,1997-10-30,9.750,1997-10-30,10.7500,1997-10-30,19.1875,1997-10-30,1.5938,1997-10-30,15.5000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6825,2024-12-10,55.4900,2024-12-10,56.7500,2024-12-10,187.1900,2024-12-10,60.7300,2024-12-10,133.0700,...,2024-12-10,316.500,2024-12-10,29.6200,2024-12-10,7.8300,NaT,NaN,2024-12-10,33.6700
6826,2024-12-11,54.6900,2024-12-11,56.2900,2024-12-11,177.0000,2024-12-11,61.3300,2024-12-11,130.6100,...,2024-12-11,315.070,2024-12-11,28.0800,2024-12-11,7.4600,NaT,NaN,2024-12-11,33.9500
6827,2024-12-12,54.9900,2024-12-12,56.4200,2024-12-12,183.9500,2024-12-12,63.3000,2024-12-12,130.8900,...,2024-12-12,319.100,2024-12-12,28.4600,2024-12-12,7.5200,NaT,NaN,2024-12-12,33.9500
6828,2024-12-13,54.9200,2024-12-13,56.4800,2024-12-13,183.1000,2024-12-13,62.0600,2024-12-13,131.3400,...,2024-12-13,315.010,2024-12-13,28.4900,2024-12-13,7.4900,NaT,NaN,2024-12-13,33.9900


In [8]:
# Percent Change Function

def pct_chg(frame):

    for col in frame.columns:
        if col.endswith('Price') and pd.api.types.is_numeric_dtype(frame[col]):
            frame[col] = frame[col].pct_change(fill_method=None)
        else:
            continue

    return frame

In [9]:
# Percent Change Function Applied

s_returns_df = pct_chg(s_returns_df).dropna()
u_returns_df = pct_chg(u_returns_df).dropna()

u_returns_df

,DUK Date,DUK Price,LNT Date,LNT Price,WEC Date,WEC Price,CMS Date,CMS Price,DTE Date,DTE Price,...,AVA Date,AVA Price,BKH Date,BKH Price,D Date,D Price,AES Date,AES Price,RSG Date,RSG Price
1,1998-07-02,-0.001052,1998-07-02,0.000000,1998-07-02,-0.002040,1998-07-02,-0.001403,1998-07-02,-0.006079,...,1998-07-02,0.002762,1998-07-02,-0.005277,1998-07-02,0.000000,1998-07-02,-0.018454,1998-07-02,0.007353
2,1998-07-06,-0.003157,1998-07-06,-0.009957,1998-07-06,0.000000,1998-07-06,0.004213,1998-07-06,-0.001530,...,1998-07-06,-0.011019,1998-07-06,0.021220,1998-07-06,0.009146,1998-07-06,-0.010578,1998-07-06,0.007299
3,1998-07-07,-0.008449,1998-07-07,0.004024,1998-07-07,0.000000,1998-07-07,-0.009790,1998-07-07,-0.006125,...,1998-07-07,0.005571,1998-07-07,0.000000,1998-07-07,-0.009063,1998-07-07,-0.007126,1998-07-07,-0.014493
4,1998-07-08,-0.005325,1998-07-08,-0.002007,1998-07-08,0.000000,1998-07-08,-0.001412,1998-07-08,0.006163,...,1998-07-08,0.000000,1998-07-08,-0.002597,1998-07-08,0.003049,1998-07-08,0.017944,1998-07-08,0.007353
5,1998-07-09,-0.006424,1998-07-09,0.008032,1998-07-09,-0.010201,1998-07-09,-0.002829,1998-07-09,0.000000,...,1998-07-09,-0.008310,1998-07-09,-0.013021,1998-07-09,-0.003040,1998-07-09,0.019975,1998-07-09,0.007299
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6654,2024-12-10,-0.006705,2024-12-10,-0.003448,2024-12-10,0.000412,2024-12-10,0.000148,2024-12-10,0.001396,...,2024-12-10,-0.005612,2024-12-10,0.000000,2024-12-10,-0.010434,2024-12-10,0.002941,2024-12-10,-0.003061
6655,2024-12-11,-0.013680,2024-12-11,-0.004778,2024-12-11,-0.015852,2024-12-11,-0.012415,2024-12-11,-0.007953,...,2024-12-11,-0.014781,2024-12-11,-0.006191,2024-12-11,-0.013452,2024-12-11,-0.003666,2024-12-11,-0.006330
6656,2024-12-12,-0.003924,2024-12-12,0.000993,2024-12-12,0.002615,2024-12-12,0.003143,2024-12-12,-0.001818,...,2024-12-12,-0.004910,2024-12-12,-0.000328,2024-12-12,-0.008108,2024-12-12,-0.005151,2024-12-12,-0.001046
6657,2024-12-13,-0.003115,2024-12-13,-0.000331,2024-12-13,-0.001252,2024-12-13,0.004326,2024-12-13,0.004140,...,2024-12-13,0.006305,2024-12-13,-0.000328,2024-12-13,0.003901,2024-12-13,-0.012574,2024-12-13,-0.005520


In [10]:
def dates_col_renamer(frame):
    new_col_date_ls = []

    for col in frame.columns:
        pos = col.find(' ')
        ticker = col[:pos]
        new_col_name = f"{ticker} Earnings Date"
        new_col_date_ls.append(new_col_name)

    return new_col_date_ls

In [11]:
u_dates_df.columns = dates_col_renamer(u_dates_df)
s_dates_df.columns = dates_col_renamer(s_dates_df)

s_dates_df

,MO Earnings Date,UVV Earnings Date,HSY Earnings Date,KR Earnings Date,KMB Earnings Date,HRL Earnings Date,MCK Earnings Date,CL Earnings Date,K Earnings Date,CPB Earnings Date,...,VERU Earnings Date,JBSS Earnings Date,HAIN Earnings Date,MED Earnings Date,COR Earnings Date,SAM Earnings Date,UNFI Earnings Date,NUS Earnings Date,CALM Earnings Date,FDP Earnings Date
0,1990-07-23,1990-05-15,1990-07-18,1990-03-24,1992-04-07,1991-02-05,1991-07-08,1990-04-20,1991-01-21,1991-10-18,...,1993-02-26,1991-01-16,1994-05-13,1994-04-07,1995-02-15,1996-02-26,1996-12-03,1997-02-19,1996-12-23,1997-05-08
1,1992-04-17,1990-07-26,1991-01-28,1990-06-16,1992-01-27,1992-02-04,1992-07-13,1990-07-30,1992-01-20,1992-10-23,...,1993-05-07,1993-04-19,1994-08-29,1994-07-27,1995-04-27,1996-04-23,1997-03-03,1997-04-30,1997-03-24,1997-11-04
2,1992-07-17,1990-10-10,1991-07-17,1990-10-06,1992-04-24,1993-02-09,1992-08-04,1990-10-30,1992-04-24,1991-11-13,...,1993-08-03,1993-08-05,1994-10-27,1994-10-27,1995-07-26,1996-07-24,1997-06-02,1997-07-30,1997-07-15,1998-02-23
3,1992-10-16,1991-01-10,1992-01-26,1990-12-29,1992-07-23,1993-02-18,1993-07-12,1992-04-06,1992-07-17,1992-02-12,...,1993-11-24,1993-11-01,1995-02-02,1995-04-27,1995-11-03,1996-10-22,1997-09-08,1997-10-29,1997-09-22,1998-04-30
4,1993-01-27,1991-04-10,1992-04-27,1991-03-23,1992-10-21,1993-05-21,1992-07-21,1991-04-22,1993-01-19,1992-05-13,...,1994-02-28,1994-03-01,1995-05-04,1995-07-27,1996-01-30,1997-02-26,1997-12-08,1998-02-18,1997-12-22,1998-07-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,NaT,NaT,NaT,2024-09-12,NaT,NaT,NaT,2024-07-26,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
138,NaT,NaT,NaT,2024-12-05,NaT,NaT,NaT,2024-10-25,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
139,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
140,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


In [12]:
s_returns_df

,MO Date,MO Price,UVV Date,UVV Price,HSY Date,HSY Price,KR Date,KR Price,KMB Date,KMB Price,...,SAM Date,SAM Price,UNFI Date,UNFI Price,NUS Date,NUS Price,CALM Date,CALM Price,FDP Date,FDP Price
1,1997-10-27,-0.058446,1997-10-27,-0.009983,1997-10-27,-0.060363,1997-10-27,-0.069364,1997-10-27,-0.037736,...,1997-10-27,-0.025156,1997-10-27,-0.079365,1997-10-27,-0.027778,1997-10-27,-0.019968,1997-10-27,-0.058824
2,1997-10-28,0.062074,1997-10-28,0.011765,1997-10-28,0.039998,1997-10-28,0.006214,1997-10-28,0.012254,...,1997-10-28,0.032205,1997-10-28,0.022989,1997-10-28,0.015873,1997-10-28,0.020375,1997-10-28,-0.011719
3,1997-10-29,-0.001583,1997-10-29,0.006645,1997-10-29,0.023310,1997-10-29,0.034976,1997-10-29,0.007265,...,1997-10-29,0.000000,1997-10-29,0.028090,1997-10-29,-0.012500,1997-10-29,-0.019968,1997-10-29,0.011858
4,1997-10-30,-0.015816,1997-10-30,-0.021452,1997-10-30,0.002278,1997-10-30,0.015905,1997-10-30,-0.016827,...,1997-10-30,-0.025000,1997-10-30,-0.060109,1997-10-30,-0.028481,1997-10-30,0.040815,1997-10-30,-0.031250
5,1997-10-31,0.019293,1997-10-31,0.037099,1997-10-31,0.004545,1997-10-31,0.021529,1997-10-31,0.015892,...,1997-10-31,0.019282,1997-10-31,-0.023256,1997-10-31,-0.022801,1997-10-31,-0.009851,1997-10-31,0.040323
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6226,2022-07-25,0.017173,2022-07-25,0.004518,2022-07-25,0.004353,2022-07-25,-0.011462,2022-07-25,0.007704,...,2022-07-25,0.054679,2022-07-25,-0.001446,2022-07-25,-0.002342,2022-11-21,0.055379,2022-07-25,-0.007874
6227,2022-07-26,0.007073,2022-07-26,0.010975,2022-07-26,0.005303,2022-07-26,0.006782,2022-07-26,0.004122,...,2022-07-26,-0.041274,2022-07-26,0.009413,2022-07-26,-0.001174,2022-11-22,0.010105,2022-07-26,0.002761
6228,2022-07-27,-0.001586,2022-07-27,-0.003203,2022-07-27,0.001009,2022-07-27,-0.001738,2022-07-27,-0.014332,...,2022-07-27,0.030667,2022-07-27,0.011000,2022-07-27,0.019036,2022-11-23,-0.001755,2022-07-27,0.022023
6229,2022-07-28,-0.001588,2022-07-28,0.000714,2022-07-28,0.027677,2022-07-28,0.002177,2022-07-28,0.009163,...,2022-07-28,0.014461,2022-07-28,0.008515,2022-07-28,0.009456,2022-11-25,0.017053,2022-07-28,0.019865


In [13]:
# Calculating Variance for F-test (Staples)

s_var_ls = {}

for col in s_returns_df.columns:
    if col.endswith('Price'):
        pos = col.find(' ')
        ticker = col[:pos]
        map_name = f"{ticker} Earnings Date"
        
        if map_name in s_dates_df.columns:
            map_col = s_dates_df[map_name].dropna()

            # Filter DataFrames for earnings and non-earnings days
            earnings_filt_df = s_returns_df[s_returns_df[f"{ticker} Date"].isin(map_col)]
            nonEarnings_filt_df = s_returns_df[~s_returns_df[f"{ticker} Date"].isin(map_col)]

            # Calculate variance only if the DataFrame is not empty
            if not earnings_filt_df.empty:
                # Earnings Variance
                earn_filtered_col = earnings_filt_df[f"{ticker} Price"]
                earn_sample_var = np.var(earn_filtered_col, ddof=1)  # Sample variance
                earn_sample_var_len = len(earn_filtered_col)

            else:
                earn_sample_var = None
                earn_sample_var_len = 0

            if not nonEarnings_filt_df.empty:
                # Non-earnings Variance
                nonEarn_filtered_col = nonEarnings_filt_df[f"{ticker} Price"]
                nonEarn_sample_var = np.var(nonEarn_filtered_col, ddof=1)  # Sample variance
                nonEarn_sample_var_len = len(nonEarn_filtered_col)

            else:
                nonEarn_sample_var = None
                nonEarn_sample_var_len = 0

            # Store the results in the dictionary
            s_var_ls[ticker] = (earn_sample_var, earn_sample_var_len, nonEarn_sample_var, nonEarn_sample_var_len)

        else:
            print(f"Column '{map_name}' not found in s_dates_df.")

s_var_ls

{'MO': (0.0004513889519557978, 99, 0.0002772681700811262, 6131),
 'UVV': (0.0003680231630305735, 99, 0.0003567835146208751, 6131),
 'HSY': (0.001452539730904928, 99, 0.0002014469984529694, 6131),
 'KR': (0.00417079611476732, 98, 0.0003054950234284967, 6132),
 'KMB': (0.0011519364398533403, 99, 0.00018140499135466063, 6131),
 'HRL': (0.0012393756430666173, 99, 0.0002037802068455532, 6131),
 'MCK': (0.0015091546674143395, 98, 0.0004277837828368263, 6132),
 'CL': (0.0011845083818814125, 99, 0.00019981679302455893, 6131),
 'K': (0.001324189183621844, 99, 0.00019201390808793752, 6131),
 'CPB': (0.0016925719061126542, 99, 0.00020944792740170418, 6131),
 'WMK': (0.00030580563272997874, 98, 0.0002915609655298825, 6132),
 'VLGEA': (0.0018339718670298292, 105, 0.0005096373666791512, 6125),
 'SPB': (0.002352440877559018, 94, 0.0011059099287723574, 6136),
 'SYY': (0.0014931954347130593, 97, 0.00027373027026584835, 6133),
 'WDFC': (0.0007161152999233158, 99, 0.0003983469554818426, 6131),
 'CAG': (0

In [14]:
# Calculating the P-value of the F-test (Staples)

s_pValues = {}
s_reject_null = []

for stock in s_var_ls:
    ls = s_var_ls[stock]
    earn_var, earn_len, nonEarn_var, nonEarn_len = ls

    df1 = earn_len - 1
    df2 = nonEarn_len - 1

    F = earn_var / nonEarn_var

    p_value = 1 - f.cdf(F, df1, df2)

    s_pValues[stock] = p_value

    if p_value < 0.05:
        s_reject_null.append("Yes")
    else:
        continue

In [15]:
print(f"Percent of sample Staple stocks with significant p-values: {round(len(s_reject_null) / len(s_pValues) * 100)}%")

Percent of sample Staple stocks with significant p-values: 91%


In [16]:
# Calculating Variance for F-test (Utilities)

u_var_ls = {}

for col in u_returns_df.columns:
    if col.endswith('Price'):
        pos = col.find(' ')
        ticker = col[:pos]
        map_name = f"{ticker} Earnings Date"
        
        if map_name in u_dates_df.columns:
            map_col = u_dates_df[map_name].dropna()

            earnings_filt_df = u_returns_df[u_returns_df[f"{ticker} Date"].isin(map_col)]
            nonEarnings_filt_df = u_returns_df[~u_returns_df[f"{ticker} Date"].isin(map_col)]

            if not earnings_filt_df.empty:
                # Earnings Variance
                earn_filtered_col = earnings_filt_df[f"{ticker} Price"]
                if isinstance(earn_filtered_col, pd.Series):
                    earn_filtered_col = pd.to_numeric(earn_filtered_col, errors='coerce')
                    earn_sample_var = np.var(earn_filtered_col, ddof=1)
                    earn_sample_var_len = len(earn_filtered_col)
                else:
                    earn_sample_var = None
                    earn_sample_var_len = 0

            else:
                earn_sample_var = None
                earn_sample_var_len = 0

            if not nonEarnings_filt_df.empty:
                # Non-earnings Variance
                nonEarn_filtered_col = nonEarnings_filt_df[f"{ticker} Price"]
                if isinstance(nonEarn_filtered_col, pd.Series):
                    nonEarn_filtered_col = pd.to_numeric(nonEarn_filtered_col, errors='coerce')
                    nonEarn_sample_var = np.var(nonEarn_filtered_col, ddof=1)
                    nonEarn_sample_var_len = len(nonEarn_filtered_col)
                else:
                    nonEarn_sample_var = None
                    nonEarn_sample_var_len = 0

            else:
                nonEarn_sample_var = None
                nonEarn_sample_var_len = 0

            u_var_ls[ticker] = (earn_sample_var, earn_sample_var_len, nonEarn_sample_var, nonEarn_sample_var_len)

        else:
            print(f"Column '{map_name}' not found in u_dates_df.")

u_var_ls

{'DUK': (0.0004597515691742633, 106, 0.0002237844018035779, 6552),
 'LNT': (0.000376019821017099, 104, 0.00020001964056207296, 6554),
 'WEC': (0.00023673785180951167, 106, 0.0001646383218334595, 6552),
 'CMS': (0.000850976946935935, 106, 0.00030772995998309315, 6552),
 'DTE': (0.00028059025841722956, 106, 0.00019168857391484083, 6552),
 'PPL': (0.0008146138306999119, 110, 0.00023380016939503103, 6548),
 'AEP': (0.0006131880215074364, 104, 0.0002284587528849982, 6554),
 'OGE': (0.0006630096004047392, 106, 0.00021245920812026524, 6552),
 'PCG': (0.0014396144819255519, 105, 0.0008169011571417998, 6553),
 'EIX': (0.0005843515317134355, 106, 0.00042486223547959077, 6552),
 'UGI': (0.0004839967697146372, 107, 0.00025645746173025427, 6551),
 'CNP': (0.004714092613806891, 106, 0.0003943475894793835, 6552),
 'NEE': (None, 0, None, 0),
 'PEG': (0.000563867746256374, 105, 0.0002315962730504258, 6553),
 'ED': (0.00014030234947550274, 106, 0.00015625715195582244, 6552),
 'MDU': (0.00057338282246291

In [17]:
# Calculating the P-value of the F-test (Utilities)

u_pValues = {}
u_reject_null = []

for stock in u_var_ls:
    ls = u_var_ls[stock]
    earn_var, earn_len, nonEarn_var, nonEarn_len = ls

    if earn_var is None:
        continue
    else:
        df1 = earn_len - 1
        df2 = nonEarn_len - 1

        F = earn_var / nonEarn_var

        p_value = 1 - f.cdf(F, df1, df2)

        u_pValues[stock] = p_value

        if p_value < 0.05:
            u_reject_null.append("Yes")
        else:
            continue

In [18]:
print(f"Percent of sample Utility stocks with significant p-values: {round(len(u_reject_null) / len(u_pValues) * 100)}%")

Percent of sample Utility stocks with significant p-values: 89%
